In [107]:
import json
import networkx as nx
import dynetx as dn
import re
import numpy as np

In [108]:
#function to read .json file of the input graph
def readPreferenceGraph(filePath) :
    with open(filePath) as file :
        data = json.load(file)

    G = nx.DiGraph()

    for edge in data["edges"] :
        G.add_edge(edge["source"], edge["target"], opinion=edge["opinion"])
    
    return G

In [109]:
def calculateProbability(weight) :
    if re.match(r".*perceives.*prefers.*collaborate.*|.*perceives.*prefers.*socialize.*", weight) :
        return 0.7
    elif re.match(r".*perceives.*avoids.*collaborate.*|.*perceives.*avoids.*socialize.*", weight) :
        return 0.3
    elif re.match(r".*prefers.*collaborate.*|.*prefers.*socialize.*", weight) :
        return 1
    else :
        return 0

In [110]:
def calculateIndexes(G, n) :
    #Direct Sociometric Indexes
    Sp = [0] * n #incoming edges to a node - elections
    Pp = [0] * n #incoming edges to a node - perceptions of elections
    Sn = [0] * n #incoming edges to a node - rejections
    Pn = [0] * n #incoming edges to a node - perceptions of rejections
    Rp = [0] * n #number of cases - reciprocal elections
    Rn = [0] * n #number of cases - reciprocal rejections
    Os = [0] * n #number of cases - feeling opposition (election & rejection)
    Ep = [0] * n #outcoming edges from a node - elections
    En = [0] * n #outcoming edges from a node - rejections
    Pap = [0] * n #number of nodes - have perception of election and get election
    Pan = [0] * n #number of nodes - have perception of rejection and get rejection


    for node in G.nodes() :
        for u,v,data in G.in_edges(node, data=True) :
            if calculateProbability(data["opinion"]) == 1 :
                Sp[node] += 1
                if G.has_edge(v,u) :
                    if calculateProbability(G[v][u]["opinion"]) == 1 :
                        Rp[node] += 1
                    elif calculateProbability(G[v][u]["opinion"]) == 0 :
                        Os[node] += 1
            elif calculateProbability(data["opinion"]) == 0.7 :
                Pp[node] += 1
            elif calculateProbability(data["opinion"]) == 0.3 :
                Pn[node] += 1
            else :
                Sn[node] += 1
                if G.has_edge(v,u) :
                    if calculateProbability(G[v][u]["opinion"]) == 0 :
                        Rn[node] += 1
                    elif calculateProbability(G[v][u]["opinion"]) == 1 :
                        Os[node] += 1
        
        for u,v,data in G.out_edges(node, data=True) :
            if calculateProbability(data["opinion"]) == 1 :
                Ep[node] += 1
            elif calculateProbability(data["opinion"]) == 0 :
                En[node] += 1
            elif calculateProbability(data["opinion"]) == 0.7 :
                if G.has_edge(v,u) and calculateProbability(G[v][u]["opinion"]) == 1 :
                    Pap[node] += 1
            else :
                if G.has_edge(v,u) and calculateProbability(G[v][u]["opinion"]) == 0 :
                    Pan[node] += 1
                

    #Compound Sociometric Indexes
    #Individual
    popularity = [item / (n - 1) for item in Sp] 
    antipathy = [item / (n - 1) for item in Sn]
    affectiveConnection = [item1 / item2 for item1, item2 in zip(Rp, Sp)]
    sociometricStatus = [(item1 + item2 - item3 - item4) / (n - 1) for item1, item2, item3, item4 in zip(Sp, Pp, Sn, Pn)]
    positiveExpansion = [item/(n - 1) for item in Ep]
    negativeExpansion = [item/(n - 1) for item in En]
    realisticPerception = [(item1 + item2) / (item3 + item4) for item1, item2, item3, item4 in zip(Pap, Pan, Sp, Sn)]
    
    #Group
    association = sum(Rp) / (n* (n-1))
    dissociation = sum(Rn) / (n * (n-1))
    cohesion =  sum(Rp) / sum(Sp)
    socialIntensity = (sum(Sp) + sum(Sn)) / (n - 1)

    print("\033[1mIndividual Sociometrix Indexes\033[0m")
    print(f"Popularity: {popularity}")
    print(f"Antipathy: {antipathy}")
    print(f"Affective Connection: {affectiveConnection}")
    print(f"Sociometric Status: {sociometricStatus}")
    print(f"Positive Expansion: {positiveExpansion}")
    print(f"Negative Expansion: {negativeExpansion}")
    print(f"Realistic Perception: {realisticPerception}")
    
    print("\n\033[1mGroup Sociometrix Indexes\033[0m")
    print(f"Association: {association}")
    print(f"Dissociation: {dissociation}")
    print(f"Cohesion: {cohesion}")
    print(f"Social Intensity: {socialIntensity}")

In [111]:
def createInteractionGraph(preferenceGraph, threshold, maxInteractions, maxTimestamp) :
    n = preferenceGraph.number_of_nodes()
    activitiesCount = 6

    interactionGraph = []
    interactionGraphIntensity = [0] * (maxTimestamp + 1)
    degrees = [0] * n
   
        
    def analyzeQuiz() :        
        #dictionary {edge : probability of existing in the Preference Graph}
        edgesProbabilities = {}

        #examine each eage in Preference Graph
        #calculate probability. How strong is the connection between people?
        for u, v in preferenceGraph.edges() :
            if preferenceGraph.has_edge(v, u) :
            #(u,v) and (v,u) are checked together once
                if (v,u) in edgesProbabilities :
                    #(v,u) is already checked, so neglect (u,v)
                    continue 
                else :
                    #(u,v), (v,u) pair has not been checked
                    edgesProbabilities[(u,v)] = 0.5*(calculateProbability(preferenceGraph[u][v]["opinion"]) + calculateProbability(preferenceGraph[v][u]["opinion"]))
            else :
                edgesProbabilities[(u,v)] = calculateProbability(preferenceGraph[u][v]["opinion"])

        #find possible peers for a student
        #step 1: find connecting links with prob < threshold
        possiblePeers = [[] for _ in range(n)]       
        for edge, prob in edgesProbabilities.items() :
            if prob < threshold :
                possiblePeers[edge[0]].append(edge[1])
                possiblePeers[edge[1]].append(edge[0])

        #step 2: keep all the other nodes as possible peers to interact with
        possiblePeers = list(map(lambda item : list(set(range(n)) - set(item[1] + [item[0]])), enumerate(possiblePeers)))
        
        return possiblePeers
    
    

    def addEdgesAtTimestamp() :
        numberOfInteractions = np.random.choice([2,3])
        G = nx.Graph()
        G.add_nodes_from(range(n))

        for node in range(n) :
            #find for a node how many additional interactions could we add with respect to maxInteractions parameter
            possibleInteractions = maxInteractions - degrees[node]
            if possibleInteractions <= 0 :
                continue

            possibleInteractions = min(numberOfInteractions, possibleInteractions)
            #choose randomly from possible peers
            peers = np.random.choice(possiblePeers[node], size=possibleInteractions, replace=False)
            
            #split possible ways of interaction in sublists and choose randomly an activity from each sublist
            weights = [np.random.choice(sublist) for sublist in np.array_split(range(activitiesCount), possibleInteractions)]

            #add new interaction to the graph
            for peer, weight in zip(peers, weights) :
                G.add_edge(node, peer, interaction_index=weight)
                    
        interactionGraph.append(G)

    def calculateIntensity(time) :
        for node in interactionGraph[time].nodes() :
            count, sum = (0, 0)
            for u, v, data in interactionGraph[time].edges(node, data=True) :
                count += 1
                sum += data["interaction_index"]
            interactionGraphIntensity[time] += count*sum
        


    calculateIndexes(G=preferenceGraph, n=n)
    

    possiblePeers = analyzeQuiz()

    #interactions at timetsamp t = 0
    #interactions based on a scale-free network model with m=3
    interactionGraph.append(nx.barabasi_albert_graph(n=preferenceGraph.number_of_nodes(), m=3))
    for u, v in interactionGraph[0].edges() :
        interactionGraph[0][u][v]["interaction_index"] = np.random.choice(range(activitiesCount))
    calculateIntensity(time=0)
    
    for node in range(n) :
        degrees[node] += interactionGraph[0].degree(node) 
    

    #interactions at timestamps t >= 1
    for time in range(1, maxTimestamp+1) :
        addEdgesAtTimestamp()

        for node in range(n) :
            degrees[node] += interactionGraph[time].degree(node) 

        calculateIntensity(time=time)

        if nx.is_empty(interactionGraph[time]) :
            interactionGraph += [nx.Graph() for _ in range(maxTimestamp - time)]
            for i in range(maxTimestamp - time) :
                calculateIntensity(time=i)
            break
    
    print(f"Interaction Graph Intensity:{interactionGraphIntensity}")
    
    return interactionGraph

In [112]:
path = "./preferenceGraph.json"

options = {
    "threshold" : 0.4,
    "maxInteractions" : 24,
    "maxTimestamp" : 5
}

G = createInteractionGraph(preferenceGraph=readPreferenceGraph(path), **options)

Individual Sociometrix Indexes
Popularity: [0.04040404040404041, 0.06060606060606061, 0.08080808080808081, 0.030303030303030304, 0.030303030303030304, 0.06060606060606061, 0.04040404040404041, 0.0707070707070707, 0.030303030303030304, 0.09090909090909091, 0.020202020202020204, 0.030303030303030304, 0.020202020202020204, 0.08080808080808081, 0.04040404040404041, 0.050505050505050504, 0.13131313131313133, 0.1111111111111111, 0.08080808080808081, 0.06060606060606061, 0.08080808080808081, 0.09090909090909091, 0.08080808080808081, 0.09090909090909091, 0.09090909090909091, 0.10101010101010101, 0.0707070707070707, 0.0707070707070707, 0.04040404040404041, 0.050505050505050504, 0.06060606060606061, 0.050505050505050504, 0.06060606060606061, 0.050505050505050504, 0.06060606060606061, 0.08080808080808081, 0.0707070707070707, 0.030303030303030304, 0.06060606060606061, 0.04040404040404041, 0.050505050505050504, 0.04040404040404041, 0.030303030303030304, 0.020202020202020204, 0.050505050505050504, 0

In [113]:
print("Snapshots added...")
for i in range(6) :
    print(f"at t = {i}: {G[i]}")

Snapshots added...
at t = 0: Graph with 100 nodes and 291 edges
at t = 1: Graph with 100 nodes and 294 edges
at t = 2: Graph with 100 nodes and 285 edges
at t = 3: Graph with 100 nodes and 254 edges
at t = 4: Graph with 100 nodes and 112 edges
at t = 5: Graph with 100 nodes and 67 edges


In [114]:
activities = ["to share content, chat in a social network platform",
            "to participate in debate/group presentation & discussion",
            "to play games",
            "to study in group or work on collaborative projects",
            "to be volunteers",
            "to be teammates in a sports team"]

for i,j,k in G[1].edges(data=True) :
    print(f"({i},{j}) => interaction: {activities[k['interaction_index']]}")

(0,73) => interaction: to participate in debate/group presentation & discussion
(0,7) => interaction: to study in group or work on collaborative projects
(0,54) => interaction: to be volunteers
(0,20) => interaction: to be volunteers
(0,40) => interaction: to play games
(0,72) => interaction: to play games
(0,92) => interaction: to be teammates in a sports team
(1,53) => interaction: to participate in debate/group presentation & discussion
(1,40) => interaction: to play games
(1,83) => interaction: to be teammates in a sports team
(1,10) => interaction: to share content, chat in a social network platform
(1,17) => interaction: to be teammates in a sports team
(1,35) => interaction: to be teammates in a sports team
(1,41) => interaction: to participate in debate/group presentation & discussion
(2,44) => interaction: to participate in debate/group presentation & discussion
(2,99) => interaction: to study in group or work on collaborative projects
(2,24) => interaction: to be teammates in